In [21]:
def read_file(path):
    res = open(path, "r").readlines()
    res = [list(x.strip()) for x in res]
    return res


input = read_file("input")

In [22]:
from typing import Self


class Region:
    def __init__(self, label):
        self.label = label
        self.points = set()

    def add_point(self, x, y):
        self.points.add((x, y))

    def is_p1_adjecent_to_p2(self, p1x, p1y, p2x, p2y):
        if abs(p1x - p2x) + abs(p1y - p2y) == 1:
            return True
        else:
            return False

    def is_adjecent(self, label, x, y):
        if self.label != label:
            return False

        for px, py in self.points:
            if self.is_p1_adjecent_to_p2(px, py, x, y):
                return True

        return False

    def is_adjecent_region(self, other: Self):
        if self.label == other.label and self.points == other.points:
            return False  # the region itself is not adjecent with itself

        for p1x, p1y in self.points:
            for p2x, p2y in other.points:
                if self.is_p1_adjecent_to_p2(p1x, p1y, p2x, p2y):
                    return True

        return False

    @property
    def perimiter(self):
        perimiter = len(self.points) * 4

        for px, py in self.points:
            for px2, py2 in self.points:
                if self.is_p1_adjecent_to_p2(px, py, px2, py2):
                    perimiter -= 1

        return perimiter

    def get_edge_points(self):
        edges = set()
        directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
        for px, py in self.points:
            for dx, dy in directions:
                if (px, py) in edges:
                    continue
                elif (px + dx, py + dy) not in self.points:
                    edges.add((px, py))
                    continue
        return edges

    @property
    def area(self):
        return len(self.points)

    @property
    def sides(self):

        DIRECTIONS = {"N": (-1, 0), "S": (1, 0), "E": (0, 1), "W": (0, -1)}

        def is_edge(x: int, y: int, other_points: set[tuple[int, int]], direction: str):
            dx, dy = DIRECTIONS[direction]
            new_x, new_y = x + dx, y + dy
            if (new_x, new_y) in other_points:
                return False
            else:
                return True

        def is_line_adjecent(l1, l2):
            l1sx, l1sy = l1[0]
            l1ex, l1ey = l1[1]
            l2sx, l2sy = l2[0]
            l2ex, l2ey = l2[1]

            if abs(l1ex - l2sx) + abs(l1ey - l2sy) == 1:
                return True

            if abs(l2ex - l1sx) + abs(l2ey - l1sy) == 1:
                return True

            return False

        def adjecent_lines_exists(lines):
            for i, l1 in enumerate(lines):
                for j, l2 in enumerate(lines):
                    if i == j:
                        continue
                    if abs(l1[1][0] - l2[0][0]) + abs(l1[1][1] - l2[0][1]) == 1:
                        return True, i, j
                    if abs(l2[1][0] - l1[0][0]) + abs(l2[1][1] - l1[0][1]) == 1:
                        return True, i, j
            return False, None, None

        def join_adjecent_lines(lines):
            lines = sorted(lines, key=lambda x: (x[0][0], x[0][1], x[1][0], x[1][1]))
            while True:
                adjecent_lines_detected, i, j = adjecent_lines_exists(lines)
                if not adjecent_lines_detected:
                    break

                # Combine lines
                lines[i] = [lines[i][0], lines[j][1]]
                lines.pop(j)
            return lines

        points = self.points
        final_lines = []

        for direction in DIRECTIONS.keys():
            lines = []  # list of lists representing the ends of the line
            # direction = "W"
            if direction in ["N", "S"]:
                points = sorted(points, key=lambda x: (x[0], x[1]))
            else:
                points = sorted(points, key=lambda x: (x[1], x[0]))

            for x, y in points:
                if not is_edge(x, y, points, direction):
                    continue

                need_new_line = True
                for line_index, line in enumerate(lines):
                    start_x, start_y = line[0]
                    end_x, end_y = line[1]

                    if direction in ["N", "S"]:
                        # left end
                        if x == start_x and y == start_y - 1:
                            lines[line_index] = [(x, y), (end_x, end_y)]
                            need_new_line = False
                            break
                        # right end
                        if x == start_x and y == end_y + 1:
                            lines[line_index] = [(start_x, start_y), (x, y)]
                            need_new_line = False
                            break
                        # middle
                        if x == start_x and start_y < y < end_y:
                            need_new_line = False
                            break

                # add a new line
                if need_new_line:
                    lines.append([(x, y), (x, y)])

            lines = join_adjecent_lines(lines)
            final_lines += lines

        return len(final_lines)

    def __repr__(self):
        return f"Region label {self.label} with {len(self.points)} nodes, area {self.area}, perimeter {self.perimiter}, and side count {self.sides}. The price is {self.area * self.perimiter}. Points included: {self.points}"

In [23]:
from collections import deque

regions: list[Region]
regions = []

directions = [(-1, 0), (0, 1), (1, 0), (0, -1)]
visited = set()
queue = deque()

queue.append((0, 0))

while queue:
    x, y = queue.popleft()
    if (x, y) in visited:
        continue
    visited.add((x, y))

    need_new_region = True
    for r in regions:
        if r.is_adjecent(input[x][y], x, y):
            r.add_point(x, y)
            need_new_region = False
            break

    if need_new_region:
        new_region = Region(input[x][y])
        new_region.add_point(x, y)
        regions.append(new_region)

    # add adjecent tiles
    for dx, dy in directions:
        new_x, new_y = x + dx, y + dy
        if (
            0 <= new_x < len(input)
            and 0 <= new_y < len(input[0])
            and (new_x, new_y) not in visited
        ):
            if input[new_x][new_y] == input[x][y]:
                queue.appendleft((new_x, new_y))
            else:
                queue.append((new_x, new_y))

In [24]:
regions

[Region label S with 30 nodes, area 30, perimeter 34, and side count 16. The price is 1020. Points included: {(4, 0), (4, 6), (0, 2), (0, 5), (2, 2), (1, 0), (1, 6), (2, 5), (1, 3), (3, 0), (5, 0), (3, 6), (0, 1), (0, 7), (2, 4), (1, 2), (0, 4), (2, 1), (1, 5), (3, 2), (3, 5), (0, 0), (1, 1), (0, 3), (2, 0), (1, 4), (0, 6), (2, 3), (1, 7), (2, 6)},
 Region label K with 55 nodes, area 55, perimeter 42, and side count 28. The price is 2310. Points included: {(3, 4), (4, 3), (3, 1), (3, 7), (5, 4), (5, 1), (9, 2), (5, 7), (9, 5), (8, 3), (8, 9), (8, 6), (10, 3), (7, 4), (6, 2), (7, 1), (7, 7), (6, 5), (6, 8), (4, 2), (4, 5), (3, 3), (5, 6), (5, 3), (8, 2), (5, 9), (9, 7), (8, 5), (9, 4), (8, 8), (6, 1), (7, 0), (6, 4), (7, 3), (7, 9), (6, 7), (7, 6), (4, 1), (4, 7), (5, 2), (4, 4), (5, 5), (8, 4), (9, 3), (8, 1), (8, 7), (5, 8), (9, 6), (7, 2), (6, 0), (6, 6), (7, 5), (6, 3), (6, 9), (7, 8)},
 Region label Q with 124 nodes, area 124, perimeter 84, and side count 54. The price is 10416. Po

In [25]:
sum([x.perimiter * x.area for x in regions])

1461806

# Part 2

This is the algorithm for detecting lines for side calculation.

1. make a queue of points.
1. pop one point from the queue

1. the point is either an end of a line or makes a new line
1. after each point is added then we consolidate lines - join the lines that are now adjecent


In [26]:
sum([x.sides * x.area for x in regions])

887932